# Graduate Final Project: Minnesota Transit Tracker: Real-Time Monitoring for Commuters

Laure Briol

## Description

###### The Minnesota Transit Tracker: Real-Time Monitoring for Commuter Map aims to provide riders across Minnesota with real-time tracking capabilities. This project uses live data from public transit APIs and displays the data dynamically on a map interface. The map features the Green and Blue metro transit lines, in addition to tracking buses along their respective routes. The map uses visualizations such as the colors, symbols, and a legend to enhance the user experience. My experience in government has shown a real need for live interactive maps, as this makes planning and understanding transit much easier.

###### In terms of technical implementation, asyncio and aiohttp are utilized for asynchronous data fetching, ensuring efficient real-time updates without affecting the responsiveness of the map interface. GeoJSON data is employed to represent train lines, bus routes, and vehicle markers on the map, providing a geospatial context to the transit data. The integration of ipyleaflet allows for the creation of an interactive map interface, complete with layers for train lines, bus routes, vehicle markers, and additional map controls such as scale and popups.

## Problem Statement

###### Many existing transit maps do not provide real-time data integration or do not provide the visual representation of a real time map. A significant number of transit maps are static or provide live updates-- but with no visualizations. Commuters will hopefully be able to understand the transit systems better and have a better experience with Minnesota's public transit systems. Officials can leverage real-time traffic maps to optimize transit operations by adjusting bus routes according to traffic conditions. They can also track buses in service, monitor train station activity to identify peak times, and provide crucial information to both drivers and passengers using this map.

# Installations

In [1]:
!pip install aiohttp

Defaulting to user installation because normal site-packages is not writeable


## Import Statements

In [2]:
import asyncio
import aiohttp
import os
import geopandas as gpd
from ipyleaflet import Map, Marker, AwesomeIcon as Icon, LayerGroup, GeoJSON, ScaleControl, Popup
from ipywidgets import Button, HTML, VBox, Text, Output, HBox
import requests

## File Path

In [3]:
#checks file path before running code
#print(os.getcwd())
#defining URLs for transit data
#API URLs for train lines provided by this website: https://gisdata.mn.gov/dataset/us-mn-state-metc-trans-transit-routes
green_line_api = 'https://svc.metrotransit.org/nextrip/vehicles/902'
blue_line_api = 'https://svc.metrotransit.org/nextrip/vehicles/901'
#API URL for routing
bus_api = 'https://svc.metrotransit.org/nextrip/routes'
#load shapefiles for train lines and bus routes
green_line_shapefile = 'Lines/green_line.shp'
blue_line_shapefile = 'Lines/blue_line.shp'
transit_routes_shapefile = 'Lines/TransitRoutes.shp'

## Data Processing and Data Fetching

In [4]:
#define dictionaries to control data fetching/update for train lines and bus
#starts as False as nothing is updating yet
continue_update_train_green = {'value': False}
continue_update_train_blue = {'value': False}
continue_update_bus = {'value': False}

In [5]:
#define functions for fetching and displaying real-time transit data
#async function: used for live updating, calls api, gets each vehicle (each bus on the line), places a marker for each vehicle
#setting up the function and parameters (api_url, markers, icon_color, continue_update, vehicle_type='train')
    #continue_update: tells to start/stop updating
    #vehicle_type: train or bus (defaults to train (type=train))
async def fetch_and_display_data(api_url, markers, icon_color, continue_update, vehicle_type='train'):
    async with aiohttp.ClientSession() as session:
        #as long as its told to continue updating-- it continues to update
        while continue_update['value']:
            async with session.get(api_url) as response:
                #200 is good response from an api
                if response.status == 200:
                    #waits until response is good and then turns response into json
                    data = await response.json()
                    #removes old not updated markers
                    markers.clear_layers()
                    #data is a list of each vehicle (train/bus) and then add marker for each train/bus (row by row)
                    for vehicle in data:
                        latitude = vehicle['latitude']
                        longitude = vehicle['longitude']
                        icon = Icon(
                            name=vehicle_type,
                            marker_color=icon_color,
                            icon_color='white',
                            spin=False
                        )
                        #marker appears
                        marker = Marker(location=(latitude, longitude), draggable=False, icon=icon)
                        markers.add_layer(marker)
                else:
                    #failed to display marker or response is not == 200
                    print("Failed to retrieve data:", response.status)
                #wait ten seconds (as api does not update faster than this)
                await asyncio.sleep(10)
#starts data collection
async def start_fetching(api_url, markers, icon_color, continue_update, vehicle_type):
    #calls dictionary  and sets value to true
        #continue_update_train_green = {'value': False}
        #continue_update_train_blue = {'value': False}
        #continue_update_bus = {'value': False}
    continue_update['value'] = True
    #calls function fetch_and_display_data when ready
    await fetch_and_display_data(api_url, markers, icon_color, continue_update, vehicle_type)

#stops fetching data (sets ['value'] = False)
def stop_fetching(continue_update):
    continue_update['value'] = False

#same as fetch_and_display_data, but for bus routes  
async def update_bus_route(b, shapefile_path):
    if bus_route_input.value.isdigit():
        #reads bus shapefile
        gdf_shapefile = gpd.read_file(shapefile_path)
        #adds bus as a line
        specific_route = gdf_shapefile[gdf_shapefile['route'] == bus_route_input.value]
        #empty if no specific route (checks if its not empty)
        if not specific_route.empty:
            specific_route = specific_route.to_crs(epsg=4326)
            route_geojson = specific_route.__geo_interface__

            #clear the previous route lines before adding new ones
            bus_route_lines.clear_layers()

            #add the route as a new layer to bus_route_lines instead of bus_markers
            route_layer = GeoJSON(data=route_geojson, style={'color': 'red', 'weight': 2}, name=f'Bus Route {bus_route_input.value}')
            bus_route_lines.add_layer(route_layer)
            #api call for bus routes
            api_url = f'https://svc.metrotransit.org/nextrip/vehicles/{bus_route_input.value}'
            if 'task' in globals():
                task.cancel()
            globals()['task'] = asyncio.create_task(start_fetching(api_url, bus_markers, 'red', continue_update_bus, 'bus'))
        else:
            print(f"No route found for ID {bus_route_input.value}.")
    else:
        print("Please enter a valid bus route number.")

## Visualizations and Interactions

In [6]:
#initialize the map
map_center_latitude = 44.9778
map_center_longitude = -93.2650
m = Map(center=(map_center_latitude, map_center_longitude), zoom=12)
#add scale control to the map
scale = ScaleControl(position='bottomleft', max_width=100, metric=True, imperial=True)
m.add_control(scale)

#setup train lines on the map
def setup_train_line(line_name, color, shapefile_path, api_url, continue_update):
    markers = LayerGroup()
    m.add_layer(markers)
    #reads train lines shape (opens file)
    gdf_shapefile = gpd.read_file(shapefile_path)
    gdf_shapefile = gdf_shapefile.to_crs(epsg=4326)
    shapefile_geojson = gdf_shapefile.__geo_interface__
    #creates train lines
    line = GeoJSON(data=shapefile_geojson, style={'color': color, 'weight': 2}, name=f'{line_name} Line')
    m.add_layer(line)
    #creates sub titles
    title = HTML(f'<h3>{line_name} Line</h3>')
    #creates buttons
    start_button = Button(description='Start Updating')
    stop_button = Button(description='Stop Updating')
    #makes buttons functional
    start_button.on_click(lambda b: asyncio.create_task(start_fetching(api_url, markers, color, continue_update, 'train')))
    stop_button.on_click(lambda b: stop_fetching(continue_update))
    #displays train line widget set-up
    return VBox([title, start_button, stop_button])

#bus routes creates text box and can enter a bus route number
bus_route_input = Text(
    value='',
    placeholder='Enter Bus Route Number',
    description='Bus Route:',
    disabled=False
)

#creates layer for bus markers
bus_markers = LayerGroup()
#adds layer to map
m.add_layer(bus_markers)
#buttons
bus_start_button = Button(description='Start Updating Bus')
bus_stop_button = Button(description='Stop Updating Bus')
#title
bus_title = HTML('<h3>Bus Route</h3>')

#define legend HTML content for map visualization
legend_html_content = """
<div style='width: 150px; padding: 10px; border: 1px solid gray;'>
    <h4>Legend</h4>
    <ul style='list-style: none; padding: 0;'>
        <li><span style='height: 15px; width: 15px; background-color: green; border-radius: 50%; display: inline-block;'></span> Green Line</li>
        <li><span style='height: 15px; width: 15px; background-color: blue; border-radius: 50%; display: inline-block;'></span> Blue Line</li>
        <li><span style='height: 15px; width: 15px; background-color: red; border-radius: 50%; display: inline-block;'></span> Bus</li>
    </ul>
</div>
"""
#turns legend into html widget (makes into a file that displays as a box)
legend = HTML(legend_html_content)

#define functions for interaction and data retrieval
#shows all bus routes
def show_all_routes(button):
    #api bus routes
    response = requests.get(bus_api)
    #clears not updated bus routes
    routes_output.clear_output()
    #checks if the response is good
    if response.status_code == 200:
        #if resposne is good make into a json
        routes = response.json()
        #makes list of all routes look more user friendly in dash
        route_list = [f"{route['route_label']} : {route['route_id']}" for route in routes]
        routes_str = "; ".join(route_list)
        #prints to the widget window (not in consol)
        with routes_output:
            print(routes_str)
    else:
        with routes_output:
            print("Failed to retrieve routes.")

## Widgets Layout

In [7]:
#define widget layout and interactions
#output is widget window
routes_output = Output()
#button
show_routes_button = Button(description='Show All Routes')
#click on button
show_routes_button.on_click(show_all_routes)
#calls set_train_line and names line, give the line a color, path, url, and continues to update
green_line_widgets = setup_train_line('Green', 'green', green_line_shapefile, green_line_api, continue_update_train_green)
blue_line_widgets = setup_train_line('Blue', 'blue', blue_line_shapefile, blue_line_api, continue_update_train_blue)
bus_widgets = VBox([bus_title, bus_route_input, bus_start_button, bus_stop_button])
#new layer for bus_route_lines (draws bus line)
bus_route_lines = LayerGroup()
#adds layer to map
m.add_layer(bus_route_lines)
#buttons
bus_start_button.on_click(lambda b: asyncio.create_task(update_bus_route(b, transit_routes_shapefile)))
bus_stop_button.on_click(lambda b: stop_fetching(continue_update_bus))
#HBox displays next to eachother (bus routes and grey box for legend are side by side)
bus_and_legend_layout = HBox([bus_widgets, legend])
#title of map
map_title = HTML(
    value="<h2>Interactive Transit Map</h2>",
    placeholder='',
    description='',
)
#displays vertical (blue, green, and bus lines)
widgets_layout = VBox([
    green_line_widgets, blue_line_widgets, bus_and_legend_layout,
    show_routes_button, routes_output, map_title, m
])
#display the updated layout
display(widgets_layout)

## Challenges

###### The challenge for this project was first finding a package that supported real time data fetching (geolocation). Folium would automatically recenter the map, every single time the data was updated, meaning I had to go beyond the scope of the course (used ipyleaflet). Moreover, adding pop-up icons to the map would not be feasible, as they are not static elements. This caused glitches and disrupted the functionality of the real-time tracker. The final challenge arose from the incremental development approach used to build the map. Each stage of implementation, such as adding the Green Line, then the Blue Line, buses, widgets, etc., led to new functionalities working and others breaking. Despite the challenges, this approach was chosen to enhance the code, make the code more efficient (save time computationally), and readable.

## Soulutions

###### The next phase of the "Minnesota Transit Tracker: Real Time Monitoring for Commuters" project aims to enhance the existing metro transit trip planner. Building upon the current functionalities of this notebook, the upcoming phase will prioritize routes that require the least number of transfers and cover the shortest distance. Additionally, users will have the opportunity to interact with the trip planner by inputting their start and end points, allowing for a more personalized and efficient commuting experience. These enhancements not only improve the usability of the transit system, but also contribute to promoting eco-friendly transportation choices.